# Recording Viewer

Show the content of a robot recording.

In [ ]:
from pathlib import Path

import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact, Layout

from flow_control.demo.simple_loader_robotio import SimpleLoader

root_dir = Path("../tmp_experiment/")
recordings = sorted(root_dir.iterdir())

loaders = [SimpleLoader(r) for r in recordings]

print("Number of recordings:", len(recordings))
print("first", recordings[0])
print("last ", recordings[-1])

## Simple Examples

actions and keyframes might not be present

In [ ]:
sl = loaders[0]
frame_index = 0

rgb = sl.get_image(0)
depth = sl.get_depth(0)
print("image shape:", rgb.shape)

action = sl.get_action(12)
print("action:", action)

kf = sl.get_keyframes()
print("keyframes:", kf)

In [ ]:
from skimage.transform import resize
from PIL import Image

seg = np.array(sl.get_segmentation(0), dtype=float)

tmp = resize(seg, (8,8))
#tmp = np.asarray(Image.fromarray(seg).resize((8,8),resample=Image.Resampling.BICUBIC))
print(tmp.max())
print(tmp.sum())
pos = plt.imshow(tmp)
plt.colorbar(pos)
plt.show()

## Show RGB Frames

In [ ]:
# Plot the demonstrations
%matplotlib notebook
fig, ax = plt.subplots(1,figsize=(8, 6))
fig.suptitle("Recording Frames")
ax.set_axis_off()
image_h = ax.imshow(loaders[0].get_image(0))
plt.show()

def update(demo_index, frame_index):
    demo_len = loaders[demo_index].get_len()
    if frame_index >= demo_len:
        print(f"invalid frame index: {frame_index}, demo length: {demo_len}")
        return
    image = loaders[demo_index].get_image(frame_index)
    image_h.set_data(image)
    try:
        info = loaders[demo_index].get_info(frame_index)
        print(info["wp_name"])
    except AttributeError:
        pass
    fig.canvas.draw_idle()
    
slider_w = widgets.IntSlider(min=0, max=len(recordings)-1, step=1, value=0,
                             layout=Layout(width='70%'))
slider_i = widgets.IntSlider(min=0, max=200-1, step=1, value=0,
                             layout=Layout(width='70%'))

interact(update, demo_index=slider_w, frame_index=slider_i)

## Show TCP Positions

In [ ]:
%matplotlib notebook
demo_index = 0
demo_len = loaders[demo_index].get_len()

grip_w  = [loaders[demo_index].get_gripper_width(f) for f in range(demo_len)]
tcp_positions = [loaders[demo_index].get_tcp_pose(f) for f in range(demo_len)]
tcp_positions = np.array(tcp_positions)
tcp_x = tcp_positions[:,0,3]
tcp_y = tcp_positions[:,1,3]
tcp_z = tcp_positions[:,2,3]

fig, (ax, ax2) = plt.subplots(2, 1)
fig.suptitle("Robot Positions")
line = ax.imshow(loaders[demo_index].get_image(0))
ax.set_axis_off()
ax2.plot(tcp_x, label="x")
ax2.plot(tcp_y, label="y")
ax2.plot(tcp_z, label="z")
ax2.plot(grip_w, label="g")
ax2.set_ylabel("value")
ax2.set_xlabel("frame number")
vline = ax2.axvline(x=2, color="k")
ax2.legend()

def update(w):
    vline.set_data([w, w], [0, 1])
    line.set_data(loaders[demo_index].get_image(w))
    fig.canvas.draw_idle()
slider_w = widgets.IntSlider(min=0, max=demo_len-1, step=1, value=0,
                             layout=Layout(width='70%'))
interact(update, w=slider_w)

## Show Depth

In [ ]:
# Plot the demonstrations
%matplotlib notebook
fig, (ax, ax2) = plt.subplots(1,2,figsize=(8, 6))
fig.suptitle("Recording Depth")
ax.set_axis_off()
ax2.set_axis_off()
image_h = ax.imshow(loaders[0].get_image(0))
image_d = ax2.imshow(loaders[0].get_depth(0))
plt.show()

def update(demo_index, frame_index):
    demo_len = loaders[demo_index].get_len()
    if frame_index >= demo_len:
        print(f"invalid frame index: {frame_index}, demo length: {demo_len}")
        return
    image = loaders[demo_index].get_image(frame_index)
    image_h.set_data(image)
    depth = loaders[demo_index].get_depth(frame_index)
    image_d.set_data(depth)
    try:
        info = loaders[demo_index].get_info(frame_index)
        print(info["wp_name"])
    except AttributeError:
        pass
    fig.canvas.draw_idle()
    
slider_w = widgets.IntSlider(min=0, max=len(recordings)-1, step=1, value=0,
                             layout=Layout(width='70%'))
slider_i = widgets.IntSlider(min=0, max=200-1, step=1, value=0,
                             layout=Layout(width='70%'))

interact(update, demo_index=slider_w, frame_index=slider_i)

# Evaluate Statistics

In [ ]:
rewards = [sl.get_reward() for sl in loaders]
print("mean success rate", np.mean(rewards))

lenghts = [sl.get_len() for sl in loaders]
print("mean length", np.mean(lenghts))